## Get a filtration from Gudhi

In [1]:
import gudhi as gd
import tadasets
sphere = tadasets.dsphere(n=400, d=1, r=1, noise=0, seed=42)
alpha = gd.AlphaComplex(points=sphere)
st = alpha.create_simplex_tree()
filtration = list(st.get_filtration())

## Convert it to boundary map data

In [2]:
from persistent_laplacians.filtrations import simplices_by_dimension, compute_boundary_matrices, filtration_hash_map

unique_filtration_values = sorted(list(set([f for (_, f) in filtration])))
simplices_by_dim, simplices_by_dim_only_filt = simplices_by_dimension(filtration)
boundary_matrices = compute_boundary_matrices(simplices_by_dim)
boundary_maps_index_dict = filtration_hash_map(filtration, simplices_by_dim_only_filt)

In [4]:
boundary_matrices

{1: {'n_rows': 400,
  'n_cols': 797,
  'data': array([ 1., -1.,  1., ..., -1.,  1., -1.], shape=(1594,)),
  'rows': array([170, 115, 223, ...,  56, 360,  24], shape=(1594,)),
  'cols': array([  0,   0,   1, ..., 795, 796, 796], shape=(1594,))},
 2: {'n_rows': 797,
  'n_cols': 398,
  'data': array([ 1., -1.,  1., ...,  1., -1.,  1.], shape=(1194,)),
  'rows': array([400,   2,  22, ...,  31, 796, 230], shape=(1194,)),
  'cols': array([  0,   0,   0, ..., 397, 397, 397], shape=(1194,))}}

## Compute homology using persistent laplacians
For now, compute only homology (using SVD)

In [3]:
from persistent_laplacians import persistent_laplacians as pl
result = pl.process_tda(
    boundary_matrices,
    boundary_maps_index_dict
)

## Barcodes 

In [4]:
from persistent_laplacians.barcodes import compute_barcodes
print({k: v for k,v in result[1].items() if v > 0})
barcodes = compute_barcodes(result, unique_filtration_values)
print(barcodes)
barcodes = [
    (q, (unique_filtration_values[i], unique_filtration_values[j]))
    for q in barcodes.keys()
    for (i, j) in barcodes[q].keys()
]
sorted(barcodes)

{(38, 50): 1, (43, 50): 1, (46, 53): 3, (30, 49): 1, (39, 51): 1, (33, 46): 1, (33, 38): 1, (30, 40): 1, (31, 39): 1, (44, 55): 1, (31, 44): 1, (30, 51): 1, (35, 45): 1, (46, 56): 1, (36, 51): 1, (42, 44): 1, (45, 49): 3, (51, 51): 5, (34, 56): 1, (33, 51): 1, (51, 54): 2, (30, 30): 1, (40, 50): 1, (37, 45): 1, (34, 49): 1, (43, 55): 1, (33, 35): 1, (39, 56): 1, (32, 37): 1, (37, 41): 1, (38, 46): 1, (30, 55): 1, (45, 47): 3, (44, 54): 2, (34, 45): 1, (41, 52): 1, (49, 52): 4, (37, 56): 1, (36, 43): 1, (49, 51): 5, (38, 48): 1, (40, 54): 1, (47, 55): 1, (40, 56): 1, (32, 55): 1, (30, 36): 1, (32, 32): 2, (31, 49): 1, (39, 42): 1, (45, 53): 3, (35, 37): 1, (32, 44): 1, (31, 37): 1, (36, 55): 1, (35, 44): 1, (50, 53): 3, (46, 52): 4, (39, 40): 1, (42, 51): 1, (38, 41): 1, (43, 52): 1, (33, 41): 1, (50, 54): 2, (49, 49): 6, (32, 47): 1, (35, 42): 1, (33, 53): 1, (41, 43): 1, (35, 55): 1, (30, 47): 1, (39, 52): 1, (30, 31): 1, (31, 45): 1, (39, 39): 1, (31, 33): 1, (38, 40): 1, (52, 54): 2

[(1, (0.09938374319148695, 1.0000000000000697)),
 (1, (0.9999999999993767, 0.9999999999996827)),
 (1, (0.9999999999998734, 0.9999999999999882)),
 (1, (0.9999999999999973, 1.0000000000000027)),
 (1, (0.9999999999999978, 1.0000000000000016)),
 (1, (0.9999999999999991, 1.0000000000000007)),
 (1, (0.9999999999999998, 0.9999999999999999)),
 (1, (0.9999999999999998, 1.0000000000000004))]

## Verify with Gudhi

In [5]:
gudhi_persistence = [(q, (i, j)) for (q, (i,j)) in st.persistence() if q > 0]
print(sorted(gudhi_persistence))
sorted(gudhi_persistence) == sorted(barcodes)

[(1, (0.09938374319148695, 1.0000000000000697)), (1, (0.9999999999993767, 0.9999999999996827)), (1, (0.9999999999998734, 0.9999999999999882)), (1, (0.9999999999999973, 1.0000000000000027)), (1, (0.9999999999999978, 1.0000000000000016)), (1, (0.9999999999999991, 1.0000000000000007)), (1, (0.9999999999999998, 0.9999999999999999)), (1, (0.9999999999999998, 1.0000000000000004))]


True